<a href="https://colab.research.google.com/github/iyline-sigey/PREDICTIVE-ANALYSIS-ON-REMOTE-LEARNING/blob/Modelling/Random_Forest%2CKNN%2CGradient_Boosted_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
#Importing libraries
import pandas as pd
import numpy as np
#for metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding
df_clean = pd.read_csv('/content/modeling_data.csv')

# **Random Forest**

In [25]:
# Separating the target from our data
ft = (df_clean.drop(['class'],axis=1)).columns
X = df_clean['clean_tweet'].values
y = df_clean['class'].values

#  Split the data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [26]:
# set a vocabulary size. This is the maximum number of words that can be used.
vocabulary_size = 10000

# create the tokenizer that comes with Keras.
tokenizer = Tokenizer(num_words=vocabulary_size)
tokenizer.fit_on_texts(X_train)

# convert the texts to sequences.
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_test)

max_words = 5000
max_len = 200

X_train_seq_padded = pad_sequences(X_train_seq, maxlen=200)
X_val_seq_padded  = pad_sequences(X_val_seq, maxlen=200)

In [27]:
#instantiate the model 
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error,mean_squared_error, accuracy_score, classification_report, f1_score,r2_score

forest = RandomForestClassifier(random_state=0)

# Train it on our training set.
forest.fit(X_train_seq_padded , y_train)

# Predict based on the model we've trained
y_pred_ftrain = forest.predict(X_train_seq_padded)
y_pred = forest.predict(X_val_seq_padded)


print("Accuracy Score test: ",accuracy_score(y_test, y_pred))
print('R-squared test:',r2_score(y_test, y_pred))  
print('Mean Squared Error test:', mean_squared_error(y_test, y_pred))
print('\n')
print(classification_report(y_test,y_pred))


Accuracy Score test:  0.9354838709677419
R-squared test: 0.7119628339140535
Mean Squared Error test: 0.06451612903225806


              precision    recall  f1-score   support

           0       0.95      0.86      0.90        21
           1       0.93      0.98      0.95        41

    accuracy                           0.94        62
   macro avg       0.94      0.92      0.93        62
weighted avg       0.94      0.94      0.93        62



# **KNN**

In [41]:
#we are going to use k of 5
from sklearn.neighbors import KNeighborsClassifier
cl=KNeighborsClassifier(n_neighbors=5)
cl.fit(X_train_seq_padded,y_train)
#we will predict our model
y_pred =cl.predict(X_val_seq_padded)

In [42]:
#checking metrics for accuracy
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[13  8]
 [10 31]]
              precision    recall  f1-score   support

           0       0.57      0.62      0.59        21
           1       0.79      0.76      0.77        41

    accuracy                           0.71        62
   macro avg       0.68      0.69      0.68        62
weighted avg       0.72      0.71      0.71        62



In [43]:
#we will tune our parameter using gridsearch
from sklearn.model_selection import GridSearchCV
from sklearn import neighbors
params = {'n_neighbors':[2,3,4,5,6,7,8,9]}

knn = neighbors.KNeighborsClassifier()

cl_g = GridSearchCV(knn, params, cv=5)
cl_g.fit(X_train_seq_padded,y_train)
cl_g.best_params_
#the best parameter for our model is 9

{'n_neighbors': 9}

In [48]:
knn

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [46]:
#training our model using k as 9

from sklearn.neighbors import KNeighborsClassifier
cl_gs=KNeighborsClassifier(n_neighbors=9)
cl_gs.fit(X_train_seq_padded,y_train)

#predicting our model
y_pred_gs=cl_gs.predict(X_val_seq_padded)

In [47]:
#checking metrics for our models accuracy
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred_gs))
print(classification_report(y_test, y_pred_gs))

[[16  5]
 [14 27]]
              precision    recall  f1-score   support

           0       0.53      0.76      0.63        21
           1       0.84      0.66      0.74        41

    accuracy                           0.69        62
   macro avg       0.69      0.71      0.68        62
weighted avg       0.74      0.69      0.70        62



# **Gradient Boost Classifier**

In [50]:
from sklearn.ensemble import GradientBoostingClassifier


gbr = GradientBoostingClassifier(n_estimators=200, learning_rate=0.4, max_depth=2)

#training our model.
gbr = gbr.fit(X_train_seq_padded,y_train)

#predicting.
y_pred_gbr = gbr.predict(X_val_seq_padded)

In [51]:
#checking metrics for our models accuracy
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[13  8]
 [10 31]]
              precision    recall  f1-score   support

           0       0.57      0.62      0.59        21
           1       0.79      0.76      0.77        41

    accuracy                           0.71        62
   macro avg       0.68      0.69      0.68        62
weighted avg       0.72      0.71      0.71        62

